## Sketch to Image
1. Load data - just images - tensorflow dataset tfds google images
1. create sketch for each image - this will be the input (X) and the original image will be the output (Y)
1. Pass input sketch through an Encoder (resnet trained over imagenet - Frozen) to get a vector
1. Pass the vector through Generator (trainable)
1. Pass generated image through discriminator (Trainable)
1. train GAN this way

# import libraries


In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import cv2

# define parameters

In [ ]:
LOSS = 'binary_crossentropy'
OPTIMIZER_D = tf.keras.optimizers.RMSprop(lr=0.003, clipvalue=1.0, decay=6e-8)
OPTIMIZER_A = tf.keras.optimizers.RMSprop(lr=0.002, clipvalue=1.0, decay=6e-8)
METRICS = 'accuracy'

INPUT_LENGTH = 128

EPOCHS = 5# 1000
BATCH_SIZE = 64

INPUT_SHAPE = (28, 28, 1)

# load data

In [ ]:
input_data = np.expand_dims(tf.keras.datasets.mnist.load_data()[0][0], axis=-1).astype('float')
print(input_data.shape)

In [ ]:
def img_to_sketch(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    invert = cv2.bitwise_not(gray)
    blur = cv2.GaussianBlur(invert, (21, 21), 0)
    invertedblur = cv2.bitwise_not(blur)
    sketch = cv2.divide(gray, invertedblur, scale=256.0)
    return sketch

# define model

### encoder

In [ ]:
def encoder():
    return tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)

### generator

In [ ]:
def generator():
    dim = INPUT_SHAPE[0]//4
    depth = 256
    model = tf.keras.models.Sequential()
    model.add(Layers.Dense(dim*dim*depth, input_dim=INPUT_LENGTH))
    model.add(Layers.LeakyReLU())
    #model.add(Layers.BatchNormalization(momentum=0.9))
    model.add(Layers.Reshape((dim, dim, depth)))
    model.add(Layers.Conv2DTranspose(depth//4, 3, padding='same'))
    model.add(Layers.LeakyReLU())
    #model.add(Layers.BatchNormalization(momentum=0.9))
    model.add(Layers.UpSampling2D(2))
    model.add(Layers.Conv2DTranspose(depth//16, 3, padding='same'))
    model.add(Layers.LeakyReLU())
    #model.add(Layers.BatchNormalization(momentum=0.9))
    model.add(Layers.UpSampling2D(2))
    model.add(Layers.Conv2DTranspose(depth//64, 3, padding='same'))
    model.add(Layers.LeakyReLU())
    #model.add(Layers.BatchNormalization(momentum=0.9))
    model.add(Layers.Conv2DTranspose(INPUT_SHAPE[2], 3, activation='tanh', padding='same'))
    model.summary()
    return model

### discriminator

In [ ]:
def discriminator():
    model = tf.keras.models.Sequential()
    model.add(Layers.Conv2D(32, 3, padding='same', input_shape=INPUT_SHAPE))
    model.add(Layers.LeakyReLU())
    model.add(Layers.Dropout(0.4))
    model.add(Layers.Conv2D(64, 3, padding='same'))
    model.add(Layers.LeakyReLU())
    model.add(Layers.Dropout(0.4))
    model.add(Layers.Conv2D(128, 3, padding='same'))
    model.add(Layers.LeakyReLU())
    model.add(Layers.Dropout(0.4))
    model.add(Layers.MaxPooling2D(2))
    model.add(Layers.Conv2D(64, 3, padding='same'))
    model.add(Layers.LeakyReLU())
    model.add(Layers.Flatten())
    model.add(Layers.Dense(128))
    model.add(Layers.LeakyReLU())
    model.add(Layers.Dense(1, activation='sigmoid'))
    model.summary()
    return model

### generator model

In [ ]:
def gen_model(enc, gen):
    model = tf.keras.models.Sequential()
    model.add(enc(trainable=False))
    model.add(gen)
    return model

### discriminator model

In [ ]:
def disc_model(discr):
    model = tf.keras.models.Sequential()
    model.add(discr)
    model.compile(loss=LOSS, optimizer=OPTIMIZER_D, metrics=METRICS)
    return model

### adversarial model

In [ ]:
def adversarial_model(gen, discr):
    model = tf.keras.models.Sequential()
    model.add(gen)
    model.add(discr(trainable=False))
    model.compile(loss=LOSS, optimizer=OPTIMIZER_A, metrics=METRICS)
    model.summary()
    return model

# Full model

In [ ]:
encdr = encoder()
genrt = generator()
discrim = discriminator()

gen = gen_model(encdr, genrt)
discr = disc_model(discrim)
advr = adversarial_model(gen, discrim) 

In [ ]:
def get_input(batch_size):
    return input_data[np.random.randint(0, input_data.shape[0], size=batch_size), :, :, :].map(lambda img : encdr(img_to_sketch(img)))

In [ ]:
def get_noise(batch_size):
    return np.random.uniform(-1.0, 1.0, size=[batch_size, INPUT_LENGTH])

In [ ]:
def get_pos_data(batch_size):
    return input_data[np.random.randint(0, input_data.shape[0], size=batch_size), :, :, :]

In [ ]:
def get_neg_data(batch_size):
    images = gen.predict(get_input(batch_size))
    return images

In [ ]:
def get_train_labels(batch_size):
    labels = np.ones([2*batch_size, 1])
    labels[batch_size:] = 0
    return labels

# Train model

In [ ]:
def train_discr():
    data_pos = get_pos_data(BATCH_SIZE)
    data_neg = get_neg_data(BATCH_SIZE)
    data = np.append(data_pos, data_neg, axis=0)
    labels = get_train_labels(BATCH_SIZE)
    return discr.train_on_batch(data, labels)

In [ ]:
def train_advr():
    noise = get_input(BATCH_SIZE)
    labels = np.ones([BATCH_SIZE, 1])
    return advr.train_on_batch(noise, labels)

In [ ]:
def train_GAN():
    for i in range(EPOCHS):
        loss_discr = train_discr()
        loss_advr = train_advr()
        log_mesg = "%d: [D loss: %f, acc: %f]" % (i, loss_discr[0], loss_discr[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, loss_advr[0], loss_advr[1])
        print(log_mesg)

In [ ]:
train_GAN()

# Generate new images

In [ ]:
num_images = 10
noise = get_input(num_images)
generated_images = gen.predict(noise)

# View generated images

In [ ]:
plt.imshow(np.squeeze(generated_images[np.random.randint(num_images)]), cmap='gray')